In [11]:
import sys 
sys.path.append("/Users/johnzhou/research/decision-making")

from src.data.experiments import Experiment
from src.utils import build_config
from src.data.real_datasets import DynamicForagingDataset
import numpy as np
from src.features.build_features import compute_foraging_efficiency
from src.models.fit_curves import normalize_block_ala_le2022, average_blocks, sigmoid, driver_func, \
    pad_ragged_blocks
from src.utils import blockify, build_config
from scipy.optimize import minimize

import matplotlib.pyplot as plt

In [41]:
def convert_real_blocks(real_blocks, real_correct_side):
    real_blocks = np.array(real_blocks)
    unique, counts = np.unique(real_blocks, return_counts=True)
    blocks = []
    if real_correct_side[0] == 1:
        side = "LEFT"
    else:
        side = "RIGHT"
    for num_trials in counts:
        blocks.append((side, 1.0, num_trials))
        if side == "LEFT":
            side = "RIGHT"
        else:
            side = "LEFT"
    return blocks

def convert_real_actions(real_actions):
    for i, act in enumerate(real_actions):
        if np.isnan(act):
            real_actions[i] = 1
    return [action - 1 for action in real_actions]

In [54]:
import os
# assign directory
directory = "/Users/johnzhou/research/decision-making/data/DynamicForaging/"
 
# iterate over files in
# that directory
filenames = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        filenames.append(f)
print(filenames)

['/Users/johnzhou/research/decision-making/data/DynamicForaging/MR17_DynamicForaging_20221006_160204.mat', '/Users/johnzhou/research/decision-making/data/DynamicForaging/MR16_DynamicForaging_20221011_132928.mat', '/Users/johnzhou/research/decision-making/data/DynamicForaging/MR16_DynamicForaging_20221006_152352.mat', '/Users/johnzhou/research/decision-making/data/DynamicForaging/MR15_DynamicForaging_20221007_160954.mat', '/Users/johnzhou/research/decision-making/data/DynamicForaging/MR17_DynamicForaging_20221007_164236.mat', '/Users/johnzhou/research/decision-making/data/DynamicForaging/MR16_DynamicForaging_20221012_110414.mat', '/Users/johnzhou/research/decision-making/data/DynamicForaging/MR18_DynamicForaging_20221011_141239.mat', '/Users/johnzhou/research/decision-making/data/DynamicForaging/MR15_DynamicForaging_20221011_132938.mat', '/Users/johnzhou/research/decision-making/data/DynamicForaging/MR15_DynamicForaging_20221010_135246.mat', '/Users/johnzhou/research/decision-making/dat

In [56]:
real_params = []

for filename in filenames:
    try:
        real_expt = DynamicForagingDataset(filename)
    except ValueError:
        print("fail")
        continue
    # print(real_expt.num_trials)
    # print(real_expt.block)
    # print(real_expt.correct_side)
    blocks = convert_real_blocks(real_expt.block, real_expt.correct_side)
    actions = convert_real_actions(real_expt.response_side)
    rewards = real_expt.rewarded

    E = compute_foraging_efficiency(actions, rewards)
    blocked_actions = blockify(blocks, actions)
    normalized_actions = [normalize_block_ala_le2022(blocked_actions[block_idx], blocks[block_idx][0])
                          for block_idx in range(len(blocks))]
    for i in range(len(blocks)):
        if blocks[i][2] < 15:
            print(f"too short, only {blocks[i][2]}")
            continue
        x_obs = range(15)
        y_obs = normalized_actions[i][:15]

        x0 = np.array([0.1, 1, 11])
        res = scipy.optimize.minimize(driver_func, x0, args=(x_obs, y_obs), bounds=((0, 0.5), (0, float('inf')), (0, float('inf'))))
#         print(res.x)
        real_params.append((*res.x, E))
#         plt.plot(x_obs, sigmoid(x_obs, *res.x), 'r-',
#                  label='fit: eps=%5.3f, k=%5.3f, x0=%5.3f' % tuple(res.x))
#         plt.legend()
#         plt.scatter(x_obs, y_obs)
#         plt.show()

too short, only 8
too short, only 6
too short, only 10
too short, only 8
too short, only 2
too short, only 1
fail
too short, only 8
too short, only 6


In [57]:
real_params = np.array(real_params)
print(real_params.shape)
np.save("/Users/johnzhou/research/decision-making/data/DynamicForaging/real_data.npy",
       real_params)

(220, 4)
